Extract baseline eeg data function

In [10]:
import mne
import pandas as pd
from tqdm import tqdm
import warnings
import os
import re
warnings.filterwarnings("ignore", category=DeprecationWarning)

def extract_baseline_eeg_data(path_2_csv_files, path_2_save_baseline_file, labelsequence=1, bad_files=[]):
    """ 

    Objective :
        Extract baseline data from raw EEG data which is in csv format. \n
        Every raw csv file of EEG data must contain 48 markers in total (opening & closing). \n
        Basically, there are 24 markers. However, Baseline data is in the first 6 markers. \n
        (12 markers if including opening & closing markers). \n


    Parameters :
        - path_2_csv_files : path to raw EEG file (csv format). \n
        - path_2_save_baseline_file : path to save extracted baseline data of EEG (output file in *.fif format). \n
        - labelsequence (opt) : order of label sequence, in this case is 1 by default. \n
        - bad_files (opt) : raw EEG file(s) that want to be skipped to be processed by the script.

    Output :
        EEG file in *.fif format with the name formatting : Subject no_EyeCondition__TrainingCondition_HandCondition_raw.fif
        For instance, "EEG-S01-averted_left_tracking_raw". There are 6 files in total for each participant.

        REMEMBER : All resulted files will be in "AVERTED" condition since the baseline condition is in AVERTED condition.

    """
    

    list_file_names = []
    full_path_2_each_file = []

    for file in os.listdir(path_2_csv_files):

        if file.endswith(".csv"):

            if (file in bad_files):

                # Skip the bad file to be processed
                print(f"Skipped bad file : {file}")
                continue

            # Populate all file names only
            list_file_names.append(file)
            list_file_names.sort()


        for i in tqdm(range(len(list_file_names)), desc="In progress"):

            try:
                labelsequence = int(labelsequence)

            except IOError as err_filename:
                print("The format of file name is not correct or file doesn't exist \nThe format must be 'EEG-Sxx.csv' , xx=subject number ")
                raise
            except ValueError as err_integer:
                print("The labelsequence input must be integer : ", err_integer)
                raise

            else:
                if  labelsequence < 1 or labelsequence > 12:
                    print("The value for labelsequence parameter is out of range. It must be be between 1 and 12")
                    raise IndexError
                else:

                    #%% Load the data
                    fileName = list_file_names[i]
                    print("Processing file : " + list_file_names[i])

                    # Change working directory to which raw EEG files are stored
                    os.chdir(path_2_csv_files)
                    
                    # Read each file by using pandas
                    df = pd.read_csv(fileName, delimiter=',')
                    # %% Define columns for raw csv file
                    df.columns = ['Index', 'FP1', 'FP2', 'F7', 'F3', 'F4', 'F8', 'T7', 'C3', 'C4', 'T8', 'P7', 'P3', 'P4', 'P8', 'O1',
                                    'O2', 'X1', 'X2', 'X3', 'X4', 'X5',
                                    'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'Marker']

                    # Converting all markers (pandas data frame to list)
                    markers = df['Marker'].tolist()
                   
                    indicesOfMarkers = []  # Empty list to contain indices of markers
                    for i, c in enumerate(markers):
                        if "9999999" in str(c) : 
                            indicesOfMarkers.append(i) 
                    try:
                        number_markers = len(indicesOfMarkers)
                        if number_markers != 48:   # check if the number of markers = 48
                            raise ValueError("The {} file has incorrect number of markers : {} ! It MUST be 48".format(fileName,number_markers))
                    except ValueError as err_unmatch_markers:
                        print(err_unmatch_markers)
                        raise


                    # Baseline data used only averted eye condition. Since everything is turned to be white. 
                    # Participants did not see each other basically. So, different eye gaze direction does not matter.

                    # This is the order for participants with odd number, e.g., 1, 3, 5, etc..
                    oddOrder1 = ["averted_pre_right_point", "averted_pre_left_point", "averted_left_tracking", "averted_right_tracking",
                                    "averted_post_right_point", "averted_post_left_point"]
                    
                    # This is the order for participants with even number, e.g., 2, 4, 6, etc..
                    evenOrder1 = ["averted_pre_left_point", "averted_pre_right_point", "averted_right_tracking", "averted_left_tracking",
                                    "averted_post_left_point", "averted_post_right_point"]


                    listOfOrders = []
                    listOfOrders.append(oddOrder1)
                    listOfOrders.append(evenOrder1)

                    # It is used to take the above label (oddOrder1 or oddOrder2)
                    i_label_taker = 0
                    if i % 2 == 0:
                        # Even number
                        i_label_taker = 0
                    else:
                        # Odd number
                        i_label_taker = 1
                    
                    chosenOrder = listOfOrders[i_label_taker]

                    ####### BASELINE DATA #######

                    # %% Get the first 12 markers' indices and extract the data
                    indicesofBaselineMarkers = indicesOfMarkers[:13]

                    # Get the first 12 markers and chunk dataframe based on those indices, and convert it into numpy array
                    # For some data, it can be 13 markers after being extracted because when we combined the data the markers of beginning are 
                    # right after the closing marker

                    # %% Chunk the data based on opening and closing markers and get only the 16 channels data (columns)
                    chunkedData = []
                    for i in range(0, 12, 2):

                        # Convert the data into numpy array type and microvolt (for some reason, the output of OpenBCI is not in microvolt)
                        chunkedData.append(df.iloc[indicesofBaselineMarkers[i] : indicesofBaselineMarkers[i+1], 1:17].to_numpy() * 1e-6)

                    # Create 16 channels montage 10-20 international standard
                    montage = mne.channels.make_standard_montage('standard_1020')

                    # Pick only 16 channels that are used in Cyton+Daisy OpenBCI
                    # % Create info (which is used in MNE-Python)
                    ch_names = ['FP1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T7', 'C3', 'C4', 'T8', 'P7', 'P3', 'P4', 'P8', 'O1', 'O2']
                    ch_types = ['eeg'] * 16
                    info = mne.create_info(
                        ch_names=ch_names,
                        sfreq=125,
                        ch_types=ch_types)
                    info.set_montage('standard_1020', match_case=False)

                    # Match pattern EEG-Sxx (xx = any number)
                    regex = r"\D{3}-S\d+"

                    # %% Create filenames for *.fif based on the sequence of labels above
                    # filenames_fif = []
                    extracted_file_name_4_baseline = []
                    for i in chosenOrder:
                        # filenames_fif.append(fileName[4:fileName.index(".")] + "-" + i + "_raw.fif")
                        # Find characters (file name) that matches the regex
                        extracted_file_name = re.search(regex,fileName)
                        extracted_file_name_4_baseline.append(fileName[extracted_file_name.start() : extracted_file_name.end()] + "-" + i + "_raw.fif")


                    #%% Save into *.fif files
                    for i, val in tqdm(enumerate(chunkedData), desc = "Saving process..."):
                        # Load data into MNE-Python
                        baseline_data_needs_label = mne.io.RawArray(val.transpose(), info, verbose=False)
                        # Define a folder where we want to save the baseline data
                        os.chdir(path_2_save_baseline_file) 
                        # Save the data in MNE format
                        baseline_data_needs_label.save(extracted_file_name_4_baseline[i], overwrite=True)


    print(f"All baseline files have been saved in fif format in this path {path_2_save_baseline_file}")       
            


Running the function of extract_baseline_eeg_data

In [11]:
# TODO Uncomment this when checked
labelsequence = 1

# TODO Uncomment this when checked
path_2_csv_files = "/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/"

# TODO Uncomment this when checked
path_2_save_baseline_file = "/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data"

# bad_file_list = ["EEG-S8.csv"]

extract_baseline_eeg_data(path_2_csv_files, path_2_save_baseline_file, labelsequence)

In progress: 0it [00:00, ?it/s]
In progress: 0it [00:00, ?it/s]
In progress:   0%|                                        | 0/1 [00:00<?, ?it/s]

Processing file : EEG-S01.csv


Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S01-averted_pre_left_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S01-averted_pre_left_point_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S01-averted_pre_right_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S01-averted_pre_right_point_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S01-averted_right_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S01-averted_right_tracking_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S01-averted_left_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S01-averted_left_tracking_raw.fif
[done]
Writing /hpc/igum002/codes/Hyperscan

Saving process...: 6it [00:00, 55.68it/s]
In progress:   0%|                                        | 0/2 [00:00<?, ?it/s]

Processing file : EEG-S01.csv


...

Saving process...: 6it [00:00, 57.93it/s]
In progress:  94%|████████████████████████████▏ | 15/16 [00:20<00:01,  1.35s/it]

Processing file : EEG-S16.csv


Overwriting existing file.
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S16-averted_pre_right_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S16-averted_pre_right_point_raw.fif
[done]
Overwriting existing file.
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S16-averted_pre_left_point_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S16-averted_pre_left_point_raw.fif
[done]
Overwriting existing file.
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S16-averted_left_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S16-averted_left_tracking_raw.fif
[done]
Overwriting existing file.
Writing /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data/EEG-S16-averted_right_tracking_raw.fif
Closing /hpc/igum002/codes/Hyperscanning2-redesign/d

Saving process...: 6it [00:00, 49.82it/s]
In progress: 100%|██████████████████████████████| 16/16 [00:21<00:00,  1.35s/it]

All baseline files have been saved in fif format in this path /hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_baseline_data


Extract experimental eeg data 

In [3]:
import mne
import pandas as pd
from tqdm import tqdm
import warnings
import os
import re

warnings.filterwarnings("ignore", category=DeprecationWarning)


def extract_experimental_eeg_data(
    path_2_csv_files,
    path_2_save_experimental_file,
    labelsequence_experiment: list,
    bad_files=[],
):

    """

    Objective :
        Extract experimental data from raw EEG data which is in csv format. \n
        Every raw csv file of EEG data must contain 48 markers in total (opening & closing). \n
        Basically, there are 24 markers. However, experimental data is from marker 7 to 24. \n
        (36 markers if including opening & closing markers). \n


    Parameters :
        - path_2_csv_files (str) : path to raw EEG file (csv format). \n
        - path_2_save_experimental_file (str) : path to save extracted experimental data of EEG (output file in *.fif format). \n
        - labelsequence_experiment (list) : order of label sequence. \n
        - bad_files (list) (opt) : raw EEG file(s) that want to be skipped to be processed by the script.

    Output :
        EEG file in *.fif format with the name formatting : Subject no_EyeCondition__TrainingCondition_HandCondition_raw.fif
        For instance, "EEG-S01-averted_left_tracking_raw". In total, there will be 18 files for each participant.

    """

    list_file_names = []
    full_path_2_each_file = []

    for file in os.listdir(path_2_csv_files):

        if file.endswith(".csv"):

            if file in bad_files:

                # Skip the bad file to be processed
                print(f"Skipped bad file : {file}")
                continue

            # Populate all file names only
            list_file_names.append(file)
            list_file_names.sort()

    for i in tqdm(range(len(list_file_names)), desc="In progress"):

        try:
            labelsequence = int(labelsequence_experiment[i])

        except IOError as err_filename:
            print(
                "The format of file name is not correct or file doesn't exist \nThe format must be 'EEG-Sxx.csv' , xx=subject number "
            )
            raise
        except ValueError as err_integer:
            print("The labelsequence input must be integer : ", err_integer)
            raise

        else:
            if labelsequence < 1 or labelsequence > 12:
                print(
                    "The value for labelsequence parameter is out of range. It must be be between 1 and 12"
                )
                raise IndexError
            else:

                #%% Load the data
                fileName = list_file_names[i]
                print("Processing file : " + list_file_names[i])

                # Change working directory to which raw EEG files are stored
                os.chdir(path_2_csv_files)

                # Read each file by using pandas
                df = pd.read_csv(fileName, delimiter=",")
                # %% Define columns for raw csv file
                df.columns = [
                    "Index",
                    "FP1",
                    "FP2",
                    "F7",
                    "F3",
                    "F4",
                    "F8",
                    "T7",
                    "C3",
                    "C4",
                    "T8",
                    "P7",
                    "P3",
                    "P4",
                    "P8",
                    "O1",
                    "O2",
                    "X1",
                    "X2",
                    "X3",
                    "X4",
                    "X5",
                    "X6",
                    "X7",
                    "X8",
                    "X9",
                    "X10",
                    "X11",
                    "X12",
                    "X13",
                    "X14",
                    "Marker",
                ]

                # Converting all markers (pandas data frame to list)
                markers = df["Marker"].tolist()

                indicesOfMarkers = []  # Empty list to contain indices of markers
                for i, c in enumerate(markers):
                    if "9999999" in str(c):
                        indicesOfMarkers.append(i)
                try:
                    number_markers = len(indicesOfMarkers)
                    if number_markers != 48:  # check if the number of markers = 48
                        raise ValueError(
                            "The {} file has incorrect number of markers : {} ! It MUST be 48".format(
                                fileName, number_markers
                            )
                        )
                except ValueError as err_unmatch_markers:
                    print(err_unmatch_markers)
                    raise

                # Create a list of labels for experimental data.

                # Order = 1 (Averted/Direct/Natural)
                oddOrder1 = [
                    "averted_pre_right_point",
                    "averted_pre_left_point",
                    "averted_left_tracking",
                    "averted_right_tracking",
                    "averted_post_right_point",
                    "averted_post_left_point",
                    "direct_pre_right_point",
                    "direct_pre_left_point",
                    "direct_left_tracking",
                    "direct_right_tracking",
                    "direct_post_right_point",
                    "direct_post_left_point",
                    "natural_pre_right_point",
                    "natural_pre_left_point",
                    "natural_left_tracking",
                    "natural_right_tracking",
                    "natural_post_right_point",
                    "natural_post_left_point",
                ]

                evenOrder1 = [
                    "averted_pre_left_point",
                    "averted_pre_right_point",
                    "averted_right_tracking",
                    "averted_left_tracking",
                    "averted_post_left_point",
                    "averted_post_right_point",
                    "direct_pre_left_point",
                    "direct_pre_right_point",
                    "direct_right_tracking",
                    "direct_left_tracking",
                    "direct_post_left_point",
                    "direct_post_right_point",
                    "natural_pre_left_point",
                    "natural_pre_right_point",
                    "natural_right_tracking",
                    "natural_left_tracking",
                    "natural_post_left_point",
                    "natural_post_right_point",
                ]

                # Order = 2 (Averted/Natural/Direct)
                oddOrder2 = [
                    "averted_pre_right_point",
                    "averted_pre_left_point",
                    "averted_left_tracking",
                    "averted_right_tracking",
                    "averted_post_right_point",
                    "averted_post_left_point",
                    "natural_pre_right_point",
                    "natural_pre_left_point",
                    "natural_left_tracking",
                    "natural_right_tracking",
                    "natural_post_right_point",
                    "natural_post_left_point",
                    "direct_pre_right_point",
                    "direct_pre_left_point",
                    "direct_left_tracking",
                    "direct_right_tracking",
                    "direct_post_right_point",
                    "direct_post_left_point",
                ]

                evenOrder2 = [
                    "averted_pre_left_point",
                    "averted_pre_right_point",
                    "averted_right_tracking",
                    "averted_left_tracking",
                    "averted_post_left_point",
                    "averted_post_right_point",
                    "natural_pre_left_point",
                    "natural_pre_right_point",
                    "natural_right_tracking",
                    "natural_left_tracking",
                    "natural_post_left_point",
                    "natural_post_right_point",
                    "direct_pre_left_point",
                    "direct_pre_right_point",
                    "direct_right_tracking",
                    "direct_left_tracking",
                    "direct_post_left_point",
                    "direct_post_right_point",
                ]

                # Order = 3 (Direct / Natural / Averted)
                oddOrder3 = [
                    "direct_pre_right_point",
                    "direct_pre_left_point",
                    "direct_left_tracking",
                    "direct_right_tracking",
                    "direct_post_right_point",
                    "direct_post_left_point",
                    "natural_pre_right_point",
                    "natural_pre_left_point",
                    "natural_left_tracking",
                    "natural_right_tracking",
                    "natural_post_right_point",
                    "natural_post_left_point",
                    "averted_pre_right_point",
                    "averted_pre_left_point",
                    "averted_left_tracking",
                    "averted_right_tracking",
                    "averted_post_right_point",
                    "averted_post_left_point",
                ]

                evenOrder3 = [
                    "direct_pre_left_point",
                    "direct_pre_right_point",
                    "direct_right_tracking",
                    "direct_left_tracking",
                    "direct_post_left_point",
                    "direct_post_right_point",
                    "natural_pre_left_point",
                    "natural_pre_right_point",
                    "natural_right_tracking",
                    "natural_left_tracking",
                    "natural_post_left_point",
                    "natural_post_right_point",
                    "averted_pre_left_point",
                    "averted_pre_right_point",
                    "averted_right_tracking",
                    "averted_left_tracking",
                    "averted_post_left_point",
                    "averted_post_right_point",
                ]

                # Order = 4 (Direct/Averted/Natural)
                oddOrder4 = [
                    "direct_pre_right_point",
                    "direct_pre_left_point",
                    "direct_left_tracking",
                    "direct_right_tracking",
                    "direct_post_right_point",
                    "direct_post_left_point",
                    "averted_pre_right_point",
                    "averted_pre_left_point",
                    "averted_left_tracking",
                    "averted_right_tracking",
                    "averted_post_right_point",
                    "averted_post_left_point",
                    "natural_pre_right_point",
                    "natural_pre_left_point",
                    "natural_left_tracking",
                    "natural_right_tracking",
                    "natural_post_right_point",
                    "natural_post_left_point",
                ]

                evenOrder4 = [
                    "direct_pre_left_point",
                    "direct_pre_right_point",
                    "direct_right_tracking",
                    "direct_left_tracking",
                    "direct_post_left_point",
                    "direct_post_right_point",
                    "averted_pre_left_point",
                    "averted_pre_right_point",
                    "averted_right_tracking",
                    "averted_left_tracking",
                    "averted_post_left_point",
                    "averted_post_right_point",
                    "natural_pre_left_point",
                    "natural_pre_right_point",
                    "natural_right_tracking",
                    "natural_left_tracking",
                    "natural_post_left_point",
                    "natural_post_right_point",
                ]

                # Order = 5 (Natural/Direct/Averted)
                oddOrder5 = [
                    "natural_pre_right_point",
                    "natural_pre_left_point",
                    "natural_left_tracking",
                    "natural_right_tracking",
                    "natural_post_right_point",
                    "natural_post_left_point",
                    "direct_pre_right_point",
                    "direct_pre_left_point",
                    "direct_left_tracking",
                    "direct_right_tracking",
                    "direct_post_right_point",
                    "direct_post_left_point",
                    "averted_pre_right_point",
                    "averted_pre_left_point",
                    "averted_left_tracking",
                    "averted_right_tracking",
                    "averted_post_right_point",
                    "averted_post_left_point",
                ]

                evenOrder5 = [
                    "natural_pre_left_point",
                    "natural_pre_right_point",
                    "natural_right_tracking",
                    "natural_left_tracking",
                    "natural_post_left_point",
                    "natural_post_right_point",
                    "direct_pre_left_point",
                    "direct_pre_right_point",
                    "direct_right_tracking",
                    "direct_left_tracking",
                    "direct_post_left_point",
                    "direct_post_right_point",
                    "averted_pre_left_point",
                    "averted_pre_right_point",
                    "averted_right_tracking",
                    "averted_left_tracking",
                    "averted_post_left_point",
                    "averted_post_right_point",
                ]

                # Order = 6 (Natural/Averted/Direct)
                oddOrder6 = [
                    "natural_pre_right_point",
                    "natural_pre_left_point",
                    "natural_left_tracking",
                    "natural_right_tracking",
                    "natural_post_right_point",
                    "natural_post_left_point",
                    "averted_pre_right_point",
                    "averted_pre_left_point",
                    "averted_left_tracking",
                    "averted_right_tracking",
                    "averted_post_right_point",
                    "averted_post_left_point",
                    "direct_pre_right_point",
                    "direct_pre_left_point",
                    "direct_left_tracking",
                    "direct_right_tracking",
                    "direct_post_right_point",
                    "direct_post_left_point",
                ]

                evenOrder6 = [
                    "natural_pre_left_point",
                    "natural_pre_right_point",
                    "natural_right_tracking",
                    "natural_left_tracking",
                    "natural_post_left_point",
                    "natural_post_right_point",
                    "averted_pre_left_point",
                    "averted_pre_right_point",
                    "averted_right_tracking",
                    "averted_left_tracking",
                    "averted_post_left_point",
                    "averted_post_right_point",
                    "direct_pre_left_point",
                    "direct_pre_right_point",
                    "direct_right_tracking",
                    "direct_left_tracking",
                    "direct_post_left_point",
                    "direct_post_right_point",
                ]

                # Populate all orders
                listOfOrders = []
                for i in tqdm(range(6)):
                    listOfOrders.append(eval("oddOrder" + str(i + 1)))
                    listOfOrders.append(eval("evenOrder" + str(i + 1)))

                labelsequence = labelsequence - 1
                chosenOrder = listOfOrders[labelsequence]

                # Get the experimental markers' indices and extract the data
                indicesofExperimentalMarkers = indicesOfMarkers[12:]

                # Chunk the data based on opening and closing markers and get only the 16 channels data (columns)
                chunkedData = []
                for i in range(0, len(indicesofExperimentalMarkers), 2):

                    # Convert the data into numpy array type and microvolt (for some reason, the output of OpenBCI is not in microvolt)
                    chunkedData.append(
                        df.iloc[
                            indicesofExperimentalMarkers[
                                i
                            ] : indicesofExperimentalMarkers[i + 1],
                            1:17,
                        ].to_numpy()
                        * 1e-6
                    )

                # Create 16 channels montage 10-20 international standard
                montage = mne.channels.make_standard_montage("standard_1020")

                # Pick only 16 channels that are used in Cyton+Daisy OpenBCI
                ch_names = [
                    "FP1",
                    "Fp2",
                    "F7",
                    "F3",
                    "F4",
                    "F8",
                    "T7",
                    "C3",
                    "C4",
                    "T8",
                    "P7",
                    "P3",
                    "P4",
                    "P8",
                    "O1",
                    "O2",
                ]
                ch_types = ["eeg"] * 16

                # Create info (which is used in MNE-Python)
                info = mne.create_info(ch_names=ch_names, sfreq=125, ch_types=ch_types)
                info.set_montage("standard_1020", match_case=False)

                # Match pattern EEG-Sxx (xx = any number)
                regex = r"\D{3}-S\d+"

                # Create filenames for *.fif based on the sequence of labels above
                extracted_file_name_4_experiment = []
                for i in chosenOrder:
                    extracted_file_name = re.search(regex, fileName)
                    extracted_file_name_4_experiment.append(
                        fileName[
                            extracted_file_name.start() : extracted_file_name.end()
                        ]
                        + "-"
                        + i
                        + "_raw.fif"
                    )

                # Save into *.fif files
                for i, val in tqdm(enumerate(chunkedData), desc="Saving process..."):
                    # Load data into MNE-Python
                    experiment_data_needs_label = mne.io.RawArray(
                        val.transpose(), info, verbose=False
                    )

                    # Define a folder where we want to save the experimental data
                    os.chdir(path_2_save_experimental_file)

                    # Save the data in MNE format
                    experiment_data_needs_label.save(
                        extracted_file_name_4_experiment[i], overwrite=True
                    )

    print(
        f"All experimental files have been saved in fif format in this path {path_2_save_experimental_file}"
    )


Running function of extract_experimental_eeg_data

In [5]:
# # TODO Uncomment this when checked
# labelsequence = [1,2,1,2,1,2,3,4,3,4,3,4,5,6,5,6]


# # # TODO Uncomment this when checked
# path_2_csv_files = "/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/"

# # # TODO Uncomment this when checked
# path_2_save_experimental_eeg_file = "/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data"

# bad_file_list = ["EEG-S07.csv"]

# path to csv files
path_2_csv_files = "/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/"

# path to save experimental file that has been extracted
path_2_save_experimental_eeg_file = "/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/raw_experimental_data"

# labelsequence_experiment
labelsequence = [1,2,1,2,1,2,3,4,3,4,3,4,5,6,5,6]

bad_files = []

extract_experimental_eeg_data(path_2_csv_files, path_2_save_experimental_eeg_file, labelsequence)

In [120]:
import os
path_2_csv_files = "/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG"
list_file_names = []
full_path_4_each_file = []
for (root, dirs, file) in os.walk(path_2_csv_files):
    for f in file:
        if ".csv" in f:
            # Populate all file names only
            list_file_names.append(f)
            list_file_names.sort()

            # Populate all full path of filenames
            full_path_4_each_file.append(os.path.join(root, f))
            full_path_4_each_file.sort()

    print(list_file_names)
    print(full_path_4_each_file)
    

['EEG-S1.csv', 'EEG-S2.csv', 'EEG-S3.csv', 'EEG-S4.csv', 'EEG-S5.csv', 'EEG-S6.csv', 'EEG-S7.csv', 'EEG-S8.csv']
['/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S1.csv', '/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S2.csv', '/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S3.csv', '/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S4.csv', '/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S5.csv', '/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S6.csv', '/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S7.csv', '/hpc/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S8.csv']


In [104]:
list_file_names.sort()
list_file_names

['EEG-S1.csv',
 'EEG-S2.csv',
 'EEG-S3.csv',
 'EEG-S4.csv',
 'EEG-S5.csv',
 'EEG-S6.csv',
 'EEG-S7.csv',
 'EEG-S8.csv']

In [89]:
# stringA = filenames_fif_baseline[0]
stringA = "/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S10-averted_baseline_raw.fif"
print(stringA)

# Match pattern EEG-Sx (x = any number)
regex = r"\D{3}-S\d+"


test_file_name = re.search(regex,stringA)
print(test_file_name)
print("Start index : ", test_file_name.start())
print("End index : ", test_file_name.end())
print(type(stringA[test_file_name.start():test_file_name.end()]))



/igum002/codes/Hyperscanning2-redesign/data/EEG/EEG-S10-averted_baseline_raw.fif
<re.Match object; span=(48, 55), match='EEG-S10'>
Start index :  48
End index :  55
<class 'str'>


ORIGINAL SCRIPT

In [ ]:
import mne
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
#%%
def extract_eeg_data(filename, labelsequence):
    try:
        f = open(filename)
        labelsequence = int(labelsequence)

    except IOError as err_filename:
        print("The format of file name is not correct or file doesn't exist \nThe format must be 'EEG-Sx.csv' , x=subject number ")
        raise
    except ValueError as err_integer:
        print("The labelsequence input must be integer : ", err_integer)
        raise

    else:
        if  labelsequence < 1 or labelsequence > 12:
            print("The value for labelsequence parameter is out of range. It must be be between 1 and 12")
            raise IndexError
        else:

            #%% Load the data
            # filename = "EEG-S1.csv"
            fileName = filename  # The format of file name of EEG must be like this
            print("Processing file : " + fileName)
            df = pd.read_csv(fileName, delimiter=',')
            # %% Define columns for raw csv file
            df.columns = ['Index', 'FP1', 'FP2', 'F7', 'F3', 'F4', 'F8', 'T7', 'C3', 'C4', 'T8', 'P7', 'P3', 'P4', 'P8', 'O1',
                          'O2', 'X1', 'X2', 'X3', 'X4', 'X5',
                          'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12', 'X13', 'X14', 'Marker']
            # Converting all markers (pandas data frame to list)
            markers = df['Marker'].tolist()
            # %%  Find all experimental markers and print them out.

            indicesOfMarkers = []  # Empty list to contain indices of markers
            for i, c in enumerate(markers):
                if "9999999" in str(c) : #todo untuk data EEG-S1-6.csv, tambahkan " : "
                    indicesOfMarkers.append(i) # check if the number of markers = 36
            try:
                number_markers = len(indicesOfMarkers)
                if number_markers != 48:
                    raise ValueError("The {} file has incorrect number of markers : {} ! It MUST be 36".format(fileName,number_markers))
            except ValueError as err_unmatch_markers:
                print(err_unmatch_markers)
                raise


            # %% Loop the list of labels in sequence
            # todo Create a list of labels with different sequences and put them into a list (list of list)
            # Order = 1 (Averted/Direct/Natural)
            oddOrder1 = ["averted_pre_right_point", "averted_pre_left_point", "averted_left_tracking", "averted_right_tracking",
                         "averted_post_right_point", "averted_post_left_point", "direct_pre_right_point", "direct_pre_left_point",
                         "direct_left_tracking", "direct_right_tracking", "direct_post_right_point", "direct_post_left_point",
                         "natural_pre_right_point", "natural_pre_left_point", "natural_left_tracking", "natural_right_tracking",
                         "natural_post_right_point", "natural_post_left_point"]

            evenOrder1 = ["averted_pre_left_point", "averted_pre_right_point", "averted_right_tracking", "averted_left_tracking",
                          "averted_post_left_point", "averted_post_right_point", "direct_pre_left_point", "direct_pre_right_point",
                          "direct_right_tracking", "direct_left_tracking", "direct_post_left_point", "direct_post_right_point",
                          "natural_pre_left_point", "natural_pre_right_point", "natural_right_tracking", "natural_left_tracking",
                          "natural_post_left_point", "natural_post_right_point"]

            # Order = 2 (Averted/Natural/Direct)
            oddOrder2 = ["averted_pre_right_point", "averted_pre_left_point", "averted_left_tracking", "averted_right_tracking",
                         "averted_post_right_point", "averted_post_left_point", "natural_pre_right_point", "natural_pre_left_point",
                         "natural_left_tracking", "natural_right_tracking", "natural_post_right_point", "natural_post_left_point",
                         "direct_pre_right_point", "direct_pre_left_point", "direct_left_tracking", "direct_right_tracking",
                         "direct_post_right_point", "direct_post_left_point"]

            evenOrder2 = ["averted_pre_left_point", "averted_pre_right_point", "averted_right_tracking", "averted_left_tracking",
                          "averted_post_left_point", "averted_post_right_point", "natural_pre_left_point", "natural_pre_right_point",
                          "natural_right_tracking", "natural_left_tracking", "natural_post_left_point", "natural_post_right_point",
                          "direct_pre_left_point", "direct_pre_right_point", "direct_right_tracking", "direct_left_tracking",
                          "direct_post_left_point", "direct_post_right_point"]

            # Order = 3 (Direct / Natural / Averted)
            oddOrder3 = ["direct_pre_right_point", "direct_pre_left_point", "direct_left_tracking", "direct_right_tracking",
                         "direct_post_right_point", "direct_post_left_point", "natural_pre_right_point", "natural_pre_left_point",
                         "natural_left_tracking", "natural_right_tracking", "natural_post_right_point", "natural_post_left_point",
                         "averted_pre_right_point", "averted_pre_left_point", "averted_left_tracking", "averted_right_tracking",
                         "averted_post_right_point", "averted_post_left_point"]

            evenOrder3 = ["direct_pre_left_point", "direct_pre_right_point", "direct_right_tracking", "direct_left_tracking",
                          "direct_post_left_point", "direct_post_right_point", "natural_pre_left_point", "natural_pre_right_point",
                          "natural_right_tracking", "natural_left_tracking", "natural_post_left_point", "natural_post_right_point",
                          "averted_pre_left_point", "averted_pre_right_point", "averted_right_tracking", "averted_left_tracking",
                          "averted_post_left_point", "averted_post_right_point"]

            # Order = 4 (Direct/Averted/Natural)
            oddOrder4 = ["direct_pre_right_point", "direct_pre_left_point", "direct_left_tracking", "direct_right_tracking",
                         "direct_post_right_point", "direct_post_left_point", "averted_pre_right_point", "averted_pre_left_point",
                         "averted_left_tracking", "averted_right_tracking", "averted_post_right_point", "averted_post_left_point",
                         "natural_pre_right_point", "natural_pre_left_point", "natural_left_tracking", "natural_right_tracking",
                         "natural_post_right_point", "natural_post_left_point"]

            evenOrder4 = ["direct_pre_left_point", "direct_pre_right_point", "direct_right_tracking", "direct_left_tracking",
                          "direct_post_left_point", "direct_post_right_point", "averted_pre_left_point", "averted_pre_right_point",
                          "averted_right_tracking", "averted_left_tracking", "averted_post_left_point", "averted_post_right_point",
                          "natural_pre_left_point", "natural_pre_right_point", "natural_right_tracking", "natural_left_tracking",
                          "natural_post_left_point", "natural_post_right_point"]

            # Order = 5 (Natural/Direct/Averted)
            oddOrder5 = ["natural_pre_right_point", "natural_pre_left_point", "natural_left_tracking", "natural_right_tracking",
                         "natural_post_right_point", "natural_post_left_point", "direct_pre_right_point", "direct_pre_left_point",
                         "direct_left_tracking", "direct_right_tracking", "direct_post_right_point", "direct_post_left_point",
                         "averted_pre_right_point", "averted_pre_left_point", "averted_left_tracking", "averted_right_tracking",
                         "averted_post_right_point", "averted_post_left_point"]

            evenOrder5 = ["natural_pre_left_point", "natural_pre_right_point", "natural_right_tracking", "natural_left_tracking",
                          "natural_post_left_point", "natural_post_right_point", "direct_pre_left_point", "direct_pre_right_point",
                          "direct_right_tracking", "direct_left_tracking", "direct_post_left_point", "direct_post_right_point",
                          "averted_pre_left_point", "averted_pre_right_point", "averted_right_tracking", "averted_left_tracking",
                          "averted_post_left_point", "averted_post_right_point"]

            # Order = 6 (Natural/Averted/Direct)
            oddOrder6 = ["natural_pre_right_point", "natural_pre_left_point", "natural_left_tracking", "natural_right_tracking",
                         "natural_post_right_point", "natural_post_left_point", "averted_pre_right_point", "averted_pre_left_point",
                         "averted_left_tracking", "averted_right_tracking", "averted_post_right_point", "averted_post_left_point",
                         "direct_pre_right_point", "direct_pre_left_point", "direct_left_tracking", "direct_right_tracking",
                         "direct_post_right_point", "direct_post_left_point"]

            evenOrder6 = ["natural_pre_left_point", "natural_pre_right_point", "natural_right_tracking", "natural_left_tracking",
                          "natural_post_left_point", "natural_post_right_point", "averted_pre_left_point", "averted_pre_right_point",
                          "averted_right_tracking", "averted_left_tracking", "averted_post_left_point", "averted_post_right_point",
                          "direct_pre_left_point", "direct_pre_right_point", "direct_right_tracking", "direct_left_tracking",
                          "direct_post_left_point", "direct_post_right_point"]
            #%% Add all labels into a list

            listOfOrders = []
            for i in progressbar(range(6)):
                listOfOrders.append(eval('oddOrder' + str(i+1)))
                listOfOrders.append(eval('evenOrder' + str(i+1)))
            print("Data have been extracted from : " + fileName)
            chosenOrder = listOfOrders[labelsequence-1]
            # %% Chunk the data based on opening and closing markers and get only the 16 channels data (columns)
            chunkedData = []
            for i in range(0, 36, 2):
                # averted_pre_right_point = df.iloc[indicesOfMarkers[0] : indicesOfMarkers[1] +1, 1:17]
                # Change into numpy and convert it from uV (microvolts) / nV to V (volts)
                chunkedData.append(df.iloc[indicesOfMarkers[i]: indicesOfMarkers[i + 1] + 1, 1:17].to_numpy() * 1e-6)
            # %% Load each eye condition file into MNE Python
            # Create 16 channels montage 10-20 international standard
            montage = mne.channels.make_standard_montage('standard_1020')
            # % Create info
            # Pick only 16 channels that are used in Cyton+Daisy OpenBCI
            ch_names = ['FP1', 'Fp2', 'F7', 'F3', 'F4', 'F8', 'T7', 'C3', 'C4', 'T8', 'P7', 'P3', 'P4', 'P8', 'O1', 'O2']
            ch_types = ['eeg'] * 16
            info = mne.create_info(
                ch_names=ch_names,
                sfreq=125,
                ch_types=ch_types)
            info.set_montage('standard_1020', match_case=False)
            # %% Create filenames for *.fif based on the sequence of labels above
            filenames_fif = []
            for i in chosenOrder:
                filenames_fif.append(fileName[4:fileName.index(".")] + "-" + i + "_raw.fif")
            #%% Save into *.fif files
            for i, val in enumerate(chunkedData):
                data_need_label = mne.io.RawArray(val.transpose(), info, verbose=False)
                data_need_label.save(join(filenames_fif[i]), overwrite=True)
            # todo save it into MNE-BIDS format



# extract_eeg_data("EEG-S10.csv", 1)
